## **Handling All Imports**

---

All necessary, non-nitrates packages and modules are imported first. Afterwards, nitrates is imported along with many specific function imports from nitrates's modules.

In [46]:
import numpy as np
from astropy.io import fits
from astropy.table import Table, vstack
from astropy.wcs import WCS
import os
from scipy import optimize, stats, interpolate
from scipy.integrate import quad
import argparse
import time
import multiprocessing as mp
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
from matplotlib import cm
import sys
import pandas as pd
pd.options.display.max_columns = 250
pd.options.display.max_rows = 250
import healpy as hp
from copy import copy, deepcopy
# sys.path.append('BatML/')
import logging, traceback
import sys
#logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)

In [47]:
#%matplotlib inline
import nitrates
from nitrates.config import rt_dir, solid_angle_dpi_fname
from nitrates.lib import get_conn, det2dpi, mask_detxy, get_info_tab, get_twinds_tab, ang_sep, theta_phi2imxy, \
    imxy2theta_phi, convert_imxy2radec, convert_radec2thetaphi, convert_radec2imxy
from nitrates.response import RayTraces
from nitrates.models import Cutoff_Plaw_Flux, Plaw_Flux, get_eflux_from_model, Source_Model_InOutFoV, \
    Bkg_Model_wFlatA, CompoundModel, Point_Source_Model_Binned_Rates, im_dist
from nitrates.llh_analysis import parse_bkg_csv, LLH_webins, NLLH_ScipyMinimize_Wjacob
print(nitrates.config.NITRATES_RESP_DIR, rt_dir)

/Volumes/WD/Development/Programming/NITRATES_RESP_DIR_PIP/ /Volumes/WD/Development/Programming/NITRATES_RESP_DIR_PIP/ray_traces_detapp_npy


## **Establishing Working Directories and Connecting to Results Database**
---

The 'NITRATES_RESP_DIR' environment variable stores the path to the NITRATES_RESP_DIR_PIP directory. Set the path to where you archived your NITRATES response files directory.

In [48]:
os.environ['NITRATES_RESP_DIR'] = '/Volumes/WD/Development/Programming/NITRATES_RESP_DIR_PIP/'

The F646018360 directory is created within the NITRATES_RESP_DIR directory. Within this directory, the files contained within csv_files.tar.gz are unpacked.

In [49]:
os.system(f'cp -r ./F646018360 {nitrates.config.NITRATES_RESP_DIR}')
os.system(f'tar -xzf csv_files.tar.gz -C {os.path.join(nitrates.config.NITRATES_RESP_DIR,"F646018360/")}')

0

Set up the NITRATES_RESP_DIR directory as described in the https://github.com/Swift-BAT/NITRATES. **NITRATES_path** retrieves the path to the NITRATES_RESP_DIR directory from config.py.

**word_dir** stores the path to the subdirectory NITRATES_RESP_DIR_PIP/F646018360 which contains many of the .fits, .db files used in this notebook.

In [50]:
NITRATES_path=nitrates.config.NITRATES_RESP_DIR
work_dir = os.path.join(NITRATES_path, 'F646018360')

The output of this notebook is stored within the file, 'results.db'. In order to work within the file, a database connection is opened to allow sqlite3 to work with it. **conn** is a connection object which represents the connection to this database. 

In [51]:
conn = get_conn(os.path.join(work_dir,'results.db'))

Within the results.db database is the info tab. The info tab for this particular database stores the time start, time stop, and trigger time among other values.**info_tab** uses get_info_tab() with argument **conn** to retrieve the info tab.

In [52]:
info_tab = get_info_tab(conn)

# To view the info_tab data, uncomment the following line
#info_tab

## **Setting Up Energy Bins**
---

The energy bins are defined through two numpy arrays. **ebins0** contains the lower bounds for each bin while **ebins1** contains the upper bounds for each bin. **ebins0** is set up with energies of 15.0keV, 24.0keV, 35.0keV, 48.0keV, and 64.0keV. 

In [53]:
ebins0 = np.array([15.0, 24.0, 35.0, 48.0, 64.0])

It is appended by 5 additional points spaced out logarithmically between 84.0keV and 500.0keV. Since the detectors are only callibrated for energies up to 350keV, the last energy of 500.0keV is discarded.

In [54]:
ebins0 = np.append(ebins0, np.logspace(np.log10(84.0), np.log10(500.0), 5+1))[:-1]

The entire array is rounded to one decimal place. The last energy is discarded again since 350.0 keV is our uppermost calibrated energy. This energy will be added at the end of **ebins1** instead.

In [55]:
ebins0 = np.round(ebins0, decimals=1)[:-1]

**ebins1** is constructed from **ebins0** starting at its second element ending with an additional 350.0keV energy. 

In [56]:
ebins1 = np.append(ebins0[1:], [350.0])
nebins = len(ebins0)
print("Nebins: ", nebins)
print("ebins0: ", ebins0)
print("ebins1: ", ebins1)

Nebins:  9
ebins0:  [ 15.   24.   35.   48.   64.   84.  120.  171.5 245. ]
ebins1:  [ 24.   35.   48.   64.   84.  120.  171.5 245.  350. ]


## **Trigger Time and Setting Up Time Window**
---

The trigger time is the exact timestamp that an event of interest occurs. This timestamp is given as the time ellapsed since mission start on January 1, 2001 00:00:00 UTC. In this scenario, the event of interest is a potential GRB event.

The trigger time is contained in the **info_tab** table under column 'trigtimeMET'.

In [57]:
trigger_time = info_tab['trigtimeMET'][0]
print('trigger_time: ', trigger_time)

trigger_time:  646018383.1787


In order to capture the full evolution of the GRB, a window of time is constructed around **trigger_time**. The beginning of this window, **t_start**, occurs 1e3 seconds before **trigger_time** and the end of this window, **t_end**, occurs 1e3 seconds after **trigger_time**. 

(see section 7. Targeted GRB Search Pipeline)

In [58]:
t_end = trigger_time + 1e3
t_start = trigger_time - 1e3

## **Reading in Event Data and Designating Good Time Intervals**
---

The event data is stored in the 'filter_evdata.fits' file within **work_dir**. **ev_data** reads in the event data stored within the file.

In [59]:
evfname = os.path.join(work_dir,'filter_evdata.fits')
ev_data = fits.open(evfname)[1].data

# To view the ev_data data, uncomment the following line
ev_data

FITS_rec([(6.46018333e+08,  1346, 0, 2685, 0.0000000e+00,  25, 134,  252,  25.2, 0),
          (6.46018333e+08, 14333, 0, 2699, 5.1307690e-04, 234, 104,  229,  22.9, 0),
          (6.46018333e+08, 18679, 0, 2420, 0.0000000e+00,  68,   0,  562,  56.2, 0),
          ...,
          (6.46018533e+08, 14227, 0, 1744, 5.9490534e-04, 247, 103, 2004, 200.4, 0),
          (6.46018533e+08, 12353, 0, 2462, 8.8046494e-05, 223, 155,  621,  62.1, 0),
          (6.46018533e+08, 23871, 0, 2287, 0.0000000e+00, 115,  40,  923,  92.3, 0)],
         dtype=(numpy.record, [('TIME', '>f8'), ('DET_ID', '>i2'), ('EVENT_FLAGS', 'u1'), ('PHA', '>i2'), ('MASK_WEIGHT', '>f4'), ('DETX', '>i2'), ('DETY', '>i2'), ('PI', '>i2'), ('ENERGY', '>f8'), ('SLEW', '>i8')]))

The GTI or "Good Time Interval" are intervals of time for which the instruments are calibrated and usable for analysis. This data is stored in the filter_evdata.fits file.

**GTI_PNT** stores the good time intervals for which the craft is actively pointing at a fixed location in space. Similarly, **GTI_SLEW** stores the good time intervals for which the spacecraft is slewing to a new pointing orientation.

(see section 7.1 Data Preparation)

In [60]:
GTI_PNT = Table.read(evfname, hdu='GTI_POINTING')
GTI_SLEW = Table.read(evfname, hdu='GTI_SLEW')

## **Reading in Attitude Data**
---

attitude.fits stores data relating to where the instrument is pointed. **attfile** opens and stores the attitude data. 

In [61]:
attfile = fits.open(os.path.join(work_dir,'attitude.fits'))[1].data

# To view the attfile data, uncomment the following line
attfile

FITS_rec([(6.46000922e+08, [ 0.70613949, -0.58084469, -0.40184175,  0.05009672], [270.15215232, -30.61826799, 218.65690613], 1),
          (6.46000923e+08, [ 0.70614058, -0.58084337, -0.40184182,  0.05009609], [270.15234872, -30.61838986, 218.65681458], 1),
          (6.46000924e+08, [ 0.70616164, -0.58083113, -0.4018256 ,  0.05007119], [270.15598075, -30.62000045, 218.65231323], 1),
          ...,
          (6.46043530e+08, [-0.31367327, -0.53680109, -0.78323223,  0.00096847], [157.34551202,  29.49830241, 285.11172485], 1),
          (6.46043535e+08, [-0.3136725 , -0.53680366, -0.78323078,  0.00097094], [157.34572342,  29.49833859, 285.11141968], 1),
          (6.46043540e+08, [-0.31367247, -0.53680369, -0.78323077,  0.00097115], [157.34574487,  29.49834958, 285.11141968], 1)],
         dtype=(numpy.record, [('TIME', '>f8'), ('QPARAM', '>f8', (4,)), ('POINTING', '>f8', (3,)), ('SOURCE', 'u1')]))

From **attfile**, **att_ind** finds the appropriate table index corresponding to the trigger time. With this index, **att_quat** retrieves the attitude data in quaternionic form. 

In [62]:
att_ind = np.argmin(np.abs(attfile['TIME'] - trigger_time))
att_quat = attfile['QPARAM'][att_ind]
print('attitude quaternion: ', att_quat)

attitude quaternion:  [-0.03597053  0.2345147  -0.64420835  0.72712074]


## **Managing Defective Detectors and Masked Detectors**
---

Not all detectors within the detector array may be functioning. When downloading new TTE data from https://swift.gsfc.nasa.gov/sdc/ql, files relating to new craft attitudes and enabled detectors are also downloaded. In this notebook, the 'detmask.fits' file stores information on which pixels in the detector array are currently functional. **dmask** stores the data from the 'detmask.fits' file.

(See section 7.1 Data Preparation)

In [63]:
dmask = fits.open(os.path.join(work_dir,'detmask.fits'))[0].data

# To view the dmask data, uncomment the following line
#dmask

Within **dmask**, detectors labelled 0 are considered "good" and detectors labelled 1 are "bad". **ndets** is the total number of "good" detectors.

In [64]:
ndets = np.sum(dmask==0)
print("Ndets: ", np.sum(dmask==0))

Ndets:  14932


The positions of the detectors within the detector array are indicated by their DETX and DETY coordinates. The function, mask_detxy(), gets rid of event data for which the detectors are masked (i.e. dmask value at (DETX,DETY) is 1).

In [65]:
mask_vals = mask_detxy(dmask, ev_data)

**bl_dmask** is the matrix of **dmask** after evaluating, for each entry of **dmask**, that it is equal to 0.

In [66]:
bl_dmask = (dmask==0.)

# To view the bl_dmask data, uncomment the following line
#bl_dmask

## **Narrowing Search To Find GRB Event Data**

---

The event data is now screened for only the data that will be useful for the analysis. From **ev_data**, **bl_ev** retrieves the indices for which (in order), the event flags are 0 indicating no errors occured during the event, the energy sits between 14keV and 500keV, the mask values are 0, and the event takes place within the timeframe between **t_start** and **t_end**. 

In [67]:
bl_ev = (ev_data['EVENT_FLAGS']<1)&\
        (ev_data['ENERGY']<=500.)&(ev_data['ENERGY']>=14.)&\
        (mask_vals==0.)&(ev_data['TIME']<=t_end)&\
        (ev_data['TIME']>=t_start)

bl_ev
print("Nevents: ",np.sum(bl_ev))

Nevents:  1367885


**ev_data0** stores the event data corresponding with the indices **bl_ev**.

In [68]:
ev_data0 = ev_data[bl_ev]

# To view the ev_data0 data, uncomment the following line
#ev_data0

## **Building Up a Simulated GRB Source**
---

A GRB is simulated at specified spatial coordinates with a cutoff power law flux model. We consider a GRB with right ascension and declination of 233.117, -26.213.

In [69]:
ra, dec = 233.117, -26.213 

The **ra** and **dec** values can be converted into detector spherical coordinates (theta,phi) using the function, convert_radec2thetaphi(). The function uses the the quaternionic form of the attitude as one of its arguments.

Likewise, **ra** and **dec** can be converted into tangential plane image coordinates using the convert_radec2imxy() function. 

In [70]:
theta, phi = convert_radec2thetaphi(ra, dec, att_quat)
print(theta, phi)

imx, imy = convert_radec2imxy(ra, dec, att_quat)
print(imx, imy)

38.54132137017975 137.65241966813443
-0.5887551341212707 -0.5366203642198198


A GRB can be described using a cutoff power law model. The equation for the cutoff power law is given by equation 16 (section 6.2 On-Time Signal Optimization):

$$ f(E) = A(\frac{E}{E_{piv}})^{\gamma} exp[-\frac{(2 - \gamma)E}{E_{peak}}]$$

where the pivot energy is given by $E_{piv} = 100\textrm{keV} $, the normalization is given by $A$, the photon index is given by $\gamma$, and the cutoff value is given by the value $E_{peak}$. 

**flux_params** is a dictionary of these three parameters with values 1.0, 0.5, and 1e2keV, respectively. Cutoff_Plaw_Flux() creates a cutoff powerlaw flux model with an energy cutoff of $E_0 = 100.0keV$.

In [71]:
flux_params = {'A':1.0, 'gamma':0.5, 'Epeak':1e2}
flux_mod = Cutoff_Plaw_Flux(E0=100.0)

## **Creating a RayTraces Object**
---

A RayTraces object is created using the path to the directory /ray_traces_detapp_npy. If **rt_dir** was not set up within config.py, replace its value with a string representing the path to /ray_traces_detapp_npy. 

In [72]:
print(nitrates.config.NITRATES_RESP_DIR)
print(rt_dir)

rt_obj = RayTraces(rt_dir)

/Volumes/WD/Development/Programming/NITRATES_RESP_DIR_PIP/
/Volumes/WD/Development/Programming/NITRATES_RESP_DIR_PIP/ray_traces_detapp_npy


The ray tracing process is simulated for a source at the **imx** and **imy** coordinates. The result is an array of ray traces onto the detector plane of the illumination fractions, i.e. how much each detector "sees" this source. This array is stored into the variable, **rt**.

In [73]:
rt = rt_obj.get_intp_rt(imx, imy)
print(np.shape(rt))

# To view the rt data, uncomment the following line
#rt

(173, 286)


## **Diffuse and Point Source Models**
---

The purpose of building a model is to eventually simulate the expected number of counts detected per detector and per energy bin as a function of our set of parameters. With these expected counts, they can be compared to the actual counts detected using the likelihood analysis. The models are split between the diffuse and point source models, which shall be described below.

### **Diffuse Model**
The diffuse model describes the expected distribution of counts $\bar{\lambda}^{diff}_{ij}$ for the ith detector and jth energy bin. The diffuse model is given by equation 8 (section 4.1.1. Diffuse Model):

$$\bar{\lambda}_{ij}^{diff} = (\Omega_i\phi^{b}_{j} + r^{b}_{j})T$$

where $\Omega_i$ is the unblocked solid angle for detector i, $\phi^{b}_{j}$ is the rate per detector solid angle, and $r^{b}_{j}$ is the rate per detector. 


### **Point Source Model**
In the case of the point source model it is dependent on the source's positions, ($\phi$, $\theta$). For the expected distribution of counts $\bar{\lambda}^{PS}_{ij}$, thes model is described by equation 12 (section 4.1.2. Point Source Model):

$$\bar{\lambda}_{ij}^{PS} = T\sum_{l}DRM_{ijl}(\theta,\phi)f(E_{\gamma} = E_{\gamma,l})\Delta E_{\gamma, l}$$

where T is the exposure, $DRM_{ijl}(\theta, \phi)$ is the detector response matrix for the source position $(\theta, \phi)$, $f(E_{\gamma} = E_{\gamma,l})$ is the photon spectrum in energy bin $E_{\gamma, l}$, and $\Delta E_{\gamma, l}$ is the difference between consecutive photon energies $E_{\gamma,l}$.

## **The Source_Model_InOutFoV Class**
---
In this section, the Source_Model_InOutFoV class is explored in detail. This class handles all of the forward modelling that is used for the likelihood analysis.

A Source_Model_InOutFoV object can be used to generate Detector Plane Images. A Detector Plane Image shows the accumulation of photon counts in each detector across the detector array. For every combination of photon energy and sky location, there are response files that have been pre-calculated and stored across multiple directories. The response components for each line of sight are accessed by their theta,phi coordinates. The responses are then assembled to create a Detector Response Matrix (DRM) which is used to calculate expected photon counts for each detector and energy bin. 

### .__init__()
---
A Source_Model_InOutFoV object is constructed using the following __init__() definition:

```
def __init__(self, flux_model,
                 ebins, bl_dmask, rt_obj,
                 name='Signal', use_deriv=False,
                 use_prior=False, resp_tab_dname=None,
                 hp_flor_resp_dname=None, comp_flor_resp_dname=None):
```


The flux_model gets saved into a class variable, **self.fmodel**, while the energy bins get stored in **self.ebins**. The lower bounds and upper bounds for the energy bins are specifically stored into **self.ebins0** and **self.ebins1**. The number of energy bins gets stored in **nebins**


```
self.ebins = ebins
self.ebins0 = ebins[0]
self.ebins1 = ebins[1]
nebins = len(self.ebins0)
```

The paths to the direct response directory (RESP_TAB_DNAME), florescence response directory (HP_FLOR_RESP_DNAME), and the compton-plus-florescence response directory (COMP_FLOR_RESP_DNAME) are stored in class variables **.resp_dname**, **.flor_resp_dname**, and **.comp_flor_resp_dname**, respectively.


For the purposes of error propogation, the errors for florescence, compton and florescence, non-florescence, coded and non-coded responses are the following:

```
self.flor_err = 0.2
self.comp_flor_err = 0.16
self.non_flor_err = 0.12
self.non_coded_err = 0.1
self.coded_err = 0.05
```

The RayTraces object that was passed as an argument gets stored into a class variable, **self.rt_obj**.

The following variables are called upon when running the function **.set_theta_phi()**. The values for **._resp_phi** and **._resp_theta** are set to NaN. The update value, **._resp_update**, is set up as 5.0 degrees. Similarly for the transmission files, the values for **._trans_phi** and **._trans_theta** are set to NaN while the **.trans_update** value is set to 5e-3 degrees. 

A signal model is created using the flux model, energy bins, usable detectors, and RayTraces object that were created above.

In [74]:
%%time
sig_mod = Source_Model_InOutFoV(flux_mod, [ebins0,ebins1], bl_dmask,\
                                rt_obj, use_deriv=True)

CPU times: user 701 µs, sys: 386 µs, total: 1.09 ms
Wall time: 983 µs


### .get_batxys()

---

This function takes x and y indices from the detector coordinates given through **self.bl_dmask** and retrieves the corresponding physical coordinates within the BAT instrument coordinate system. The function makes use of the detxy2batxy(xinds, yinds) function to retrieve the coordinates. The resulting x and y coordinates are stored as class variables **.batxs** and **.batys**.

In [75]:
sig_mod.get_batxys()

# To view the .batxs and .batys, uncomment the following lines
#print(sig_mod.batxs)
#print(sig_mod.batys)

### .set_theta_phi(theta, phi)

---

This function takes in arguments for theta and phi and updates the class variables, **.theta** and **.phi**, with the new values.

These values are compared to **.resp_theta** and **.resp_phi**, which hold the values for the theta and phi values that were last used to query the response files. If the angle separation between (**.theta**, **.phi**) and (**.resp_theta**, **.resp_phi**) is larger than **._resp_update**, then the new theta and phi values are used to query new response files. 

If the angle separation is less than **._resp_update**, we must also check if the angle separation is larger than **._trans_theta**. If so, then the transmission values must also be updated as they are also queried by theta and phi values.

In [76]:
#The theta and phi values prior to running set_theta_phi()
sig_mod.set_theta_phi(theta, phi)

(0.0, 30.48, -14.117)
(0.0, 30.48, -14.117)


/Volumes/WD/Development/Programming/NITRATES_RESP_DIR_PIP/swift-bat/lib/python3.9/site-packages/nitrates/response/response.py:1499: RuntimeWarning: invalid value encountered in divide
  self.E_A0s = (self.orig_photonEs[self.Einds1] - self.photonEs) /\


(36.0, 45.0)
2.652419668134428
42.34758033186557
max rt: 0.8604
initing ResponseDPI, with fname
/Volumes/WD/Development/Programming/NITRATES_RESP_DIR_PIP/resp_tabs_ebins/drm_theta_36.0_phi_30.0_.fits
initing ResponseDPI, with fname
/Volumes/WD/Development/Programming/NITRATES_RESP_DIR_PIP/resp_tabs_ebins/drm_theta_36.0_phi_45.0_.fits
initing ResponseDPI, with fname
/Volumes/WD/Development/Programming/NITRATES_RESP_DIR_PIP/resp_tabs_ebins/drm_theta_45.0_phi_30.0_.fits
initing ResponseDPI, with fname
/Volumes/WD/Development/Programming/NITRATES_RESP_DIR_PIP/resp_tabs_ebins/drm_theta_45.0_phi_45.0_.fits


### .set_flux_params(flux_params)
---
**.set_flux_params(flux_params)** takes a list of parameters, flux_params, as an argument and updates the class variable, **.flux_param**, with the new values. The following normed dpis are also set: **.normed_photon_fluxes**, **.normed_comp_flor_rate_dpis**, **.normed_photoe_rate_dpis**, **.normed_rate_dpis**, and lastly,  **.normed_err_rate_dpis**. The normed dpis are useful for varying parameters such as the amplitude, A, without modifying the underlying dpis.




### .get_rate_dpis(params)
---
The **.get_rate_dpis(params)** function returns the dpis corresponding to the dictionary of parameters given. These parameters contain values for theta, phi, and the normalization constant, A. 

For new angles, if the angle separation is larger than **_trans_update** , then .set_theta_phi() and .set_flux_params() are run again for the new parameters. Finally, the rate_dpis are calculated after multiplying **.normed_rate_dpis** by the normalization constant, A, and returned.


In [77]:
sig_mod

## **Background Model**
---

The background model encompasses any of the non-GRB sources that show up in the image. This can include contributions from the cosmic X-rays/gamma-rays or any number of sources that are known to be sources of gamma rays. The background model consists mainly of a diffuse model due to the cosmic X-rays/gamma-rays combined with point source models for each of the known sources within the field of view.

(see section 6.1. Off-Time Background)

In general, all bright point sources are stored in https://github.com/Swift-BAT/NITRATES/blob/main/bright_src_cat.fits. For this notebook, the estimation of the background for the 4 sources relevant for this field of view is given in bkg_estimation.csv. 

The variable, **solid_ang_dpi** stores the dpi relating to the solid angle seen by each detector. 


In [78]:
bkg_fname = os.path.join(work_dir,'bkg_estimation.csv')
solid_ang_dpi = np.load(solid_angle_dpi_fname)

With bkg_estimation.csv, the function parse_bkg_csv() parses through the file in order to create point source models for each of the background sources. The function returns the following objects:

**bkg_df** - The output after running the pandas function .read_csv() on bkg_fname

**bkg_name** - Name for the background that will be used for background-related parameters.

**PSnames** - List of strings of the astronomical names of any sources of cosmic/x-rays rays found within the background 

**bkg_mod** - The background model returned by the function parse_bck_csv()

**ps_mods** - List of corresponding point source models for each of the astronomical objects listed in **PSnames**

In [79]:
bkg_df, bkg_name, PSnames, bkg_mod, ps_mods = parse_bkg_csv(bkg_fname, solid_ang_dpi,\
                    ebins0, ebins1, bl_dmask, rt_dir)

['4U 1700-377', 'GRO J1655-40', 'GX 339-4', 'Sco X-1']


The background model **bkg_mod** is added to a list of background models **bkg_mod_list**. If there are any point source models in **ps_mods** they are added to **bkg_mod_list** and has_deriv is set to False for each of them. With all the models compiled into a single list, the background model is redefined as a compound model combining all of the point source models with the original **bkg_mod**.

Later on, when a minimizer object is created, it will ask models for the Jacobian of the log likelihood function with respect to the model parameters. Since the parameters are fixed, there are no such derivatives to retrieve so .has_deriv is set to False.

In [80]:
bkg_mod.has_deriv = False
bkg_mod_list = [bkg_mod]
Nsrcs = len(ps_mods)
if Nsrcs > 0:
    bkg_mod_list += ps_mods
    for ps_mod in ps_mods:
        ps_mod.has_deriv = False
    bkg_mod = CompoundModel(bkg_mod_list)

The trigger time is used again to retrieve a dictionary of parameters at the trigger time index. **bkg_row** refers to this dictionary of parameters. The values of the parameters of the background model are retrieved from **bkg_row** and stored as another dictionary, **bkg_params**. Below is a description of the output of calling **bkg_params**.

The first set of parameters have names ending with '_bkg_rate_0', '_bkg_rate_1', etc. These rates refer to contributions to the background model from diffuse sources measured in counts per second. The number index in the parameter name refers to the corresponding energy bin for which the rate is measured (e.g the parameter '_bkg_rate_0' is the counts/s in energy bin 0 attributable to diffuse sources). 

Next are the parameter names ending with '_flat_0', '_flat_1', etc. These parameters refer to the rate per detector parameters in the diffuse model, $r^b_j$. Their values are the fraction of the above background rate taht is the result of the rate, $r^b_j$. (e.g. '_flat_0' is the fraction of the '_bkg_rate_0' attributable to the rate per detector $r^b_0$).

For each of the point sources within the background, the parameters corresponding to their imx and imy coordinates are listed along with the unmasked rates measured per energy bin.

In [81]:
tmid = trigger_time
bkg_row = bkg_df.iloc[np.argmin(np.abs(tmid - bkg_df['time']))]
bkg_params = {pname:bkg_row[pname] for pname in\
            bkg_mod.param_names}
bkg_name = bkg_mod.name
bkg_params

{'Background_bkg_rate_0': 0.0913703220701183,
 'Background_bkg_rate_1': 0.0661578002239374,
 'Background_bkg_rate_2': 0.0400898569026105,
 'Background_bkg_rate_3': 0.0394919934207801,
 'Background_bkg_rate_4': 0.0346780266835281,
 'Background_bkg_rate_5': 0.0351076584048365,
 'Background_bkg_rate_6': 0.033602492003955,
 'Background_bkg_rate_7': 0.0248610826847874,
 'Background_bkg_rate_8': 0.0176399476145076,
 'Background_flat_0': 0.0,
 'Background_flat_1': 0.0,
 'Background_flat_2': 0.1859632649428977,
 'Background_flat_3': 0.0766344244533236,
 'Background_flat_4': 0.3020855890115875,
 'Background_flat_5': 0.7611510180823338,
 'Background_flat_6': 0.8091430254293075,
 'Background_flat_7': 1.0,
 'Background_flat_8': 1.0,
 '4U 1700-377_imx': -0.0981485305770971,
 '4U 1700-377_imy': -0.4742076074486664,
 '4U 1700-377_rate_0': 0.0124901359218975,
 '4U 1700-377_rate_1': 0.0072425264886238,
 '4U 1700-377_rate_2': 0.0045295645864628,
 '4U 1700-377_rate_3': 0.0016876913514016,
 '4U 1700-377_r

The parameters in **bkg_params** are aggregated into a dictionary **pars_** such that the background rates (of the form '_bkg_rate_0') of the background model are combined with rates due to each of the source models. The naming convention combines "Background" plus any of the known sources followed by **bkg_name** and appended by the name of the parameter. For example, for the parameter "_bkg_rate_0", it is prepended by the name "Background+4U 1700-377+GRO J1655-40+GX 339-4+Sco X-1_Background".

The **pars_** dictionary additionally stores the **theta**, **phi**, and **flux_params** parameters.

In [82]:
pars_ = {}
pars_['Signal_theta'] = theta
pars_['Signal_phi'] = phi
for pname,val in list(bkg_params.items()):
    pars_[bkg_name+'_'+pname] = val
for pname,val in list(flux_params.items()):
    pars_['Signal_'+pname] = val
pars_

{'Signal_theta': 38.54132137017975,
 'Signal_phi': 137.65241966813443,
 'Background+4U 1700-377+GRO J1655-40+GX 339-4+Sco X-1_Background_bkg_rate_0': 0.0913703220701183,
 'Background+4U 1700-377+GRO J1655-40+GX 339-4+Sco X-1_Background_bkg_rate_1': 0.0661578002239374,
 'Background+4U 1700-377+GRO J1655-40+GX 339-4+Sco X-1_Background_bkg_rate_2': 0.0400898569026105,
 'Background+4U 1700-377+GRO J1655-40+GX 339-4+Sco X-1_Background_bkg_rate_3': 0.0394919934207801,
 'Background+4U 1700-377+GRO J1655-40+GX 339-4+Sco X-1_Background_bkg_rate_4': 0.0346780266835281,
 'Background+4U 1700-377+GRO J1655-40+GX 339-4+Sco X-1_Background_bkg_rate_5': 0.0351076584048365,
 'Background+4U 1700-377+GRO J1655-40+GX 339-4+Sco X-1_Background_bkg_rate_6': 0.033602492003955,
 'Background+4U 1700-377+GRO J1655-40+GX 339-4+Sco X-1_Background_bkg_rate_7': 0.0248610826847874,
 'Background+4U 1700-377+GRO J1655-40+GX 339-4+Sco X-1_Background_bkg_rate_8': 0.0176399476145076,
 'Background+4U 1700-377+GRO J1655-40+G

With the background and signal models assembled, a singular compound model can be created which combines the two models into one. **comp_mod** is a CompoundModel class object created using the **bkg_mod** and **sig_mod** variables.

In [83]:
comp_mod = CompoundModel([bkg_mod, sig_mod])

## **The Likelihood and Log-Likelihood**
---

Given a model $M(\Theta)$ with parameters $\Theta$, the number of counts $N_{ij}$, and expected counts $\theta_{ij}$, the likelihood for the ith detector and jth energy bin can be given by the Poisson likelihood, equation 4:

$$l_{ij}(M(\Theta)|N_{ij}) = Pois(N_{ij}; \lambda_{ij}) = \frac{(\lambda_{ij})^{N_{ij}}e^{-\lambda_{ij}}}{N_{ij}!}$$ 

Each of the count expectations $\lambda_{ij}$ will have an associated error given by a probability density function (PDF) in the form of a normal distribution. Given a Gaussian error $\sigma_{ij}$ and mean count expectation $\bar{\lambda}_{ij}$, the error PDF is given by equation 5:

$$P(\lambda_{ij}|M(\Theta)) = \frac{1}{\sqrt{2\pi\sigma^2_{ij}}}exp\left[-\frac{(\lambda_{ij}-\bar{\lambda}_{ij})^2}{2\sigma^2_{ij}}\right]$$

To account for this error, the likelihood function is integrated over the error PDF on $\lambda_{ij}$, shown in equation 6:

$$l_{ij}(M(\lambda_{ij}\mid M(\Theta))) = \int_{}{} Pois(N_{ij}; \lambda_{ij}) \mathcal{N}(\lambda_{ij}; \bar{\lambda}_{ij}, \sigma_{ij})d\lambda_{ij}$$

With this expression for the likelihood, the log-likelihood (LLH) is given by the equation 7:

$$LLH(M(\Theta)\mid N) = \sum_{ij}log\left[l_{ij}(M(\Theta)\mid N_{ij})\right]$$

(see section 4. A Likelihood Framework)

## **Calculating and Minimizing Log Likelihood**
---
In the minimization process, the parameters that minimize this statistical description of the observed data are presumed to be the true physical parameters.

The calculation of the log likelihood is done through the LLH_webins class object, **sig_llh_obj**, which takes in the event data (**ev_data0**), the lower and upper bounds of the energy bins (**ebins0** and **ebins1**), and the list of usable detectors (**bl_dmask**). **sig_llh_obj** calls .set_model() with argument **comp_mod** in order to utilize the compound model in the likelihood calculation. 

Note that when initializing **sig_llh_obj**, the argument has_err is set to True in order to utilize the error values set up in the compound model.

In [84]:
sig_llh_obj = LLH_webins(ev_data0, ebins0, ebins1, bl_dmask, has_err=True)
sig_llh_obj.set_model(comp_mod)

The log likelihood minimization is handled by the NLLH_ScipyMinimize_Wjacob object, **sig_miner**, which is a wrapper of the scipy minimizer object. Using the .set_llh() function, it can rely on **sig_llh_obj** to recalculate the log likelihood through variations of the parameters. 

In [85]:
sig_miner = NLLH_ScipyMinimize_Wjacob('')
sig_miner.set_llh(sig_llh_obj)

In order to cut down on the computation time, only 'Signal_A' is varied during the minimization process while the others are held constant. To keep the parameters constant, list them inside the list **fixed_pnames** with their corresponding values **fixed_vals**. A transformation on each of the parameter variables can be supplied in a list **trans**. In this scenario, all of the fixed parameteres are supplied without any transformations (i.e. None). 

The minimizer object saves these fixed parameters and transformations through the functions .set_trans() and .set_fixed_params(). Since 'Signal_A' is to be varied, we run .set_fixed_params() with the option fixed=False to inform the minimizer that this parameter is to be varied.

In [86]:
fixed_pnames = list(pars_.keys())
fixed_vals = list(pars_.values())
trans = [None for i in range(len(fixed_pnames))]
sig_miner.set_trans(fixed_pnames, trans)
sig_miner.set_fixed_params(fixed_pnames, values=fixed_vals)
sig_miner.set_fixed_params(['Signal_A'], fixed=False)

Note that in general, the flux parameters are not known a priori. For the purposes of demonstration, the source model flux parameters are set with gamma and Epeak values of 0.8 and 350.0keV, respectively. 

In [87]:
#%%time
# setting gamma and Epeak for a "known" source
flux_params['gamma'] = 0.8
flux_params['Epeak'] = 350.0
sig_mod.set_flux_params(flux_params)

Likewise, we know the exact time window of the signal to utilize. It is set with initial time **t0** and final time **t1**. The signal likelihood object can set the time and duration of the LLH analysis using .set_time().

In [88]:
# Setting initial time and final time of known GRB event
t0 = trigger_time - 0.512
t1 = t0 + 2.048
sig_llh_obj.set_time(t0, t1)

The minimization process is run using the function .minimize() which returns three objects. The first, **pars**, contains the set of parameters which minimized the likelihood. The second parameter, **nllh**, is the negative log likelihood associated with the parameters, **pars**. The last, **res**, is a return object of the Scipy minimizer object. 

In [89]:
#%%time
pars, nllh, res = sig_miner.minimize()

print(res)
print(nllh)
print(pars)

[      fun: 45894.07068100051
 hess_inv: <1x1 LbfgsInvHessProduct with dtype=float64>
      jac: array([0.00239975])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 8
      nit: 5
     njev: 8
   status: 0
  success: True
        x: array([0.006875])]
[45894.07068100051]
[[0.006875003655224625]]


The negative log likelihood is calculated for the background-only model so that it can be compared to the compound (signal+background) model. Setting the parameter 'Signal_A' to 1e-10 effectively cancels out the contribution from the signal model. Since none of the parameters are being varied, the log likelihood only needs to be calculated once using the .get_logprob() function.

In [91]:
%%time
# Evaluating log likelihood for background only model. This is accomplished by setting Signal_A to effectively zero. 
pars_['Signal_A'] = 1e-10
bkg_nllh = -sig_llh_obj.get_logprob(pars_)
print (bkg_nllh)

46038.74332720728
CPU times: user 4.94 ms, sys: 6.29 ms, total: 11.2 ms
Wall time: 9.04 ms


The likelihood ratio test statistic given by equation 13 (section 6. GRB Search):

$$\lambda  = -2[LLH(\Theta_{bkg}) - LLH(\Theta_{sig}, \Theta_{bkg})]$$

This statistic is used to quantify the preference for the signal+background model compared to the background-only model. 

In the following block, the square root of the likelihood ratio test statistic is calculated using the negative log likelihood of the background model and the minimized background+signal model. 


In [92]:
sqrtTS = np.sqrt(2.*(bkg_nllh - nllh[0]))
print (sqrtTS)

17.01015262757948


The value of **sqrtTS** indicates a strong preference for the signal+background model. As a result, we have a strong case for having detected a GRB. By comparison, a value of 8.0 is the threshold for confident detection of a GRB. At a threshold rate, the false alarm rate (FAR) is less than $10^{-4}s^{-1}$ which is equivalent to around 8.64 times a day.

## References
---
DeLaunay, J., & Tohuvavohu, A. (2021). Harvesting BAT-GUANO with NITRATES (Non-Imaging Transient Reconstruction And TEmporal Search): Detecting and localizing the faintest GRBs with a likelihood framework. doi:10.48550/ARXIV.2111.01769 
https://arxiv.org/pdf/2111.01769.pdf